In [ ]:
#written by Marina Ricci
#last revisioon 30/11/18

#purposes: handling cosmoDC2 catalog and investigating the properties of galaxy clusters

#TODO
# include dn/dz 
# include M*(z)
# include NFW parametrisation

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import GCRCatalogs
from GCR import GCRQuery
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from scipy.stats import normaltest
import matplotlib.colors as clr
%matplotlib inline

# see all available catalogs
print(sorted(GCRCatalogs.get_available_catalogs(False)))

In [ ]:
# load a calalog
catalog = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')
catalog1 = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')
print(catalog1.get_catalog_info())
# see all available quantities
print(sorted(catalog1.list_all_quantities()))

In [ ]:
cosmo = FlatLambdaCDM(H0=71, Om0=0.2648, Ob0=0.0448) #not 100% sure it is the same cosmo as in the simu

# Selection of halos with Mass >1e14 (referred to as clusters in the following) in the selected sub-sample

In [ ]:
# load quantities
data = catalog.get_quantities(['ra', 'dec','redshift','redshift_true','halo_mass', 'halo_id','is_central','mag_i_lsst','mag_Y_lsst'], native_filters=['healpix_pixel == 9813'])
#data = catalog.get_quantities(['ra', 'dec','redshift','redshift_true','halo_mass', 'halo_id','is_central','mag_i_lsst','mag_Y_lsst'],native_filters=['healpix_pixel<9821','healpix_pixel>9810'])#9807 - 9821
cluster_data = catalog1.get_quantities(['ra', 'dec','redshift','redshift_true','halo_mass', 'halo_id','is_central','mag_r_lsst','mag_i_lsst','mag_Y_lsst','Mag_true_u_lsst_z0','Mag_true_g_lsst_z0','Mag_true_i_lsst_z0'], filters=['halo_mass > 1e14'])

In [ ]:
#data = catalog.get_quantities(['ra', 'dec','redshift','redshift_true','halo_mass', 'halo_id','is_central','mag_i_lsst','mag_Y_lsst'],native_filters=['healpix_pixel<9821','healpix_pixel>9810'])#9807 - 9821

# 0 - some statistics

## - Number of objects in the two sample loaded ##

In [ ]:
print (data['is_central'].size)
print (cluster_data['is_central'].size)

## - Fraction of satellite galaxies in the two sample loaded

In [ ]:
print ("fraction of satellite galaxies", 1-np.sum(data['is_central'])/data['is_central'].size)
print ("fraction of satellite galaxies", 1-np.sum(cluster_data['is_central'])/cluster_data['is_central'].size)

## - Number of clusters

In [ ]:
mhalo0=(data['halo_mass']>1e14)*(data['is_central'])
mhalomem0=(data['halo_mass']>1e14)
print (np.sum(mhalo0))

mhalo=(cluster_data['is_central'])
print (np.sum(mhalo))

## *From now we mainly use the cluster catalog*

# 1 - Cluster mass-redshift distribution

In [ ]:
plt.figure()
plt.plot(cluster_data['redshift'][mhalo],cluster_data['halo_mass'][mhalo],'k.')
plt.yscale('log')
plt.xlabel('z')
plt.ylabel('$M_{h}$');

In [ ]:
plt.figure()
a=np.histogram(cluster_data['redshift'][mhalo],bins=25,range=[0,2.5])
plt.errorbar(a[1][:-1]+0.05,a[0]/(a[1][1]-a[1][0]),yerr=np.sqrt(a[0])/(a[1][1]-a[1][0]))
plt.xlabel('z')
plt.ylabel('dn/dz');

# **Add cosmo expectations?**

# 2 - Ra-dec distribution of the clusters

In [ ]:
plt.figure()
plt.plot(cluster_data['ra'],cluster_data['dec'],'c.',alpha=0.5,label='satellites')
plt.plot(cluster_data['ra'][mhalo],cluster_data['dec'][mhalo],'k.',label='centrals')
plt.legend(loc=3)
plt.xlabel('RA')
plt.ylabel('Dec.');

# 3 - Magnitude as a function of redshift for central and satellite galaxies

In [ ]:
plt.figure()
#plt.hist2d(data['redshift'],data['mag_i_lsst'],bins=[60,80],cmap='jet')
#plt.hist2d(cluster_data['redshift'],cluster_data['mag_i_lsst'],bins=[60,80],cmap='jet')
plt.plot(cluster_data['redshift'],cluster_data['mag_i_lsst'],'c.',alpha=0.05,label='satellites')
plt.plot(cluster_data['redshift'][mhalo],cluster_data['mag_i_lsst'][mhalo],'k.',label='centrals')

#plt.colorbar()
plt.legend(loc=4)
plt.xlabel('z')
plt.ylabel('mag i');


### *Why is there no faint galaxies in haloes at low redshift? (see also LF plot in section 6). The same is true for galaxies outside of haloes.*

# 4 - Number of member galaxies as a function of mass

In [ ]:
ngal = np.zeros(np.sum(mhalo))
for i in range(np.sum(mhalo)):
    ngal[i] = ((GCRQuery('halo_id == {}'.format(cluster_data['halo_id'][mhalo][i])).filter(cluster_data))['ra']).size
    

In [ ]:
plt.scatter(cluster_data['halo_mass'][mhalo],ngal,c=cluster_data['redshift'][mhalo],s=40,cmap='jet',vmax=1.5)
clb=plt.colorbar()
clb.set_label('z')
plt.yscale('log')
plt.xscale('log')
plt.title('Number of FoF members vs FoF halo mass')
plt.xlabel('$M_{h} (M_{\odot})$')
plt.ylabel('$N_{gal}$');
#plt.plot(cluster_data['halo_mass'][mhalo],cluster_data['halo_mass'][mhalo]/1e12,'k') #to add an arbitrary line with slope = 1 for vizualisation purpose

In [ ]:
rmax=0.75 #Mpc 

ngal_cut = np.zeros(np.sum(mhalo))

for i in range(np.sum(mhalo)):
    halo=cluster_data['halo_id'][mhalo][i]
    ra_ref = cluster_data['ra'][(cluster_data['halo_id']==halo)*mhalo]
    dec_ref = cluster_data['dec'][(cluster_data['halo_id']==halo)*mhalo]
    z_ref = cluster_data['redshift'][(cluster_data['halo_id']==halo)*mhalo]
    ngal_cut[i]=(np.sum((cosmo.kpc_proper_per_arcmin(z_ref).value*(np.sqrt((cluster_data['ra'][(cluster_data['halo_id']==halo)]-ra_ref)**2+(cluster_data['dec'][(cluster_data['halo_id']==halo)]-dec_ref)**2))*60./1e3)<rmax))
  

### *To have a representative richness-mass relation we need to count galaxies with cuts both in radius and apparent magnitude*

In [ ]:
plt.scatter(cluster_data['halo_mass'][mhalo],ngal_cut,c=cluster_data['redshift'][mhalo],s=40,cmap='jet',vmax=1.)
clb=plt.colorbar()
clb.set_label('z')
plt.yscale('log')
plt.xscale('log')
plt.title('Number of FoF members within r='+str(rmax)+'Mpc vs FoF halo mass')
plt.xlabel('$M_{h} (M_{\odot})$')
plt.ylabel('$N_{gal}$ (r<'+str(rmax)+'Mpc)');
#plt.plot(cluster_data['halo_mass'][mhalo],cluster_data['halo_mass'][mhalo]/1e12,'k') #to add an arbitrary line with slope = 1 for vizualisation purpose

### *It would be great to have physical masses and radius...*

# 5 - Galaxy projected density profiles in clusters 

In [ ]:
#compute the distance of each member to the central galaxies for the haloes in different mass/z bins

cond = [(cluster_data['redshift']<0.5)*(cluster_data['halo_mass']<1.5e14),\
        (cluster_data['redshift']>0.5)*(cluster_data['redshift']<1)*(cluster_data['halo_mass']<1.5e14),\
        (cluster_data['redshift']>1.0)*(cluster_data['redshift']<1.5)*(cluster_data['halo_mass']<1.5e14),\
        (cluster_data['redshift']<0.5)*(cluster_data['halo_mass']>2e14)*(cluster_data['halo_mass']<3e14),\
        (cluster_data['redshift']>0.5)*(cluster_data['redshift']<1)*(cluster_data['halo_mass']>2e14)*(cluster_data['halo_mass']<3e14),\
       ]
halo_list = [cluster_data['halo_id'][mhalo*cond[0]],cluster_data['halo_id'][mhalo*cond[1]],cluster_data['halo_id'][mhalo*cond[2]],cluster_data['halo_id'][mhalo*cond[3]],cluster_data['halo_id'][mhalo*cond[4]]]
dist_Mpc = [[],[],[],[],[]]
k = 0
Nclus = [len(halo_list[0]),len(halo_list[1]),len(halo_list[2]),len(halo_list[3]),len(halo_list[4])]
for j in halo_list:
    for halo in j :
        ra_ref = cluster_data['ra'][(cluster_data['halo_id']==halo)*mhalo]
        dec_ref = cluster_data['dec'][(cluster_data['halo_id']==halo)*mhalo]
        z_ref = cluster_data['redshift'][(cluster_data['halo_id']==halo)*mhalo]
        dist_Mpc[k].append(cosmo.kpc_proper_per_arcmin(z_ref).value*(np.sqrt((cluster_data['ra'][(cluster_data['halo_id']==halo)]-ra_ref)**2+(cluster_data['dec'][(cluster_data['halo_id']==halo)]-dec_ref)**2))*60./1e3)
    k = k+1
dist_Mpc[0] = np.array(np.hstack(dist_Mpc[0]))
dist_Mpc[1] = np.array(np.hstack(dist_Mpc[1]))
dist_Mpc[2] = np.array(np.hstack(dist_Mpc[2]))
dist_Mpc[3] = np.array(np.hstack(dist_Mpc[3]))
dist_Mpc[4] = np.array(np.hstack(dist_Mpc[4]))

print ("Number of clusters in each bin", Nclus)

In [ ]:
#compute the composite projected density profiles in each mass/z bin

dist_hist0=np.histogram(dist_Mpc[0][dist_Mpc[0]>0],bins=100,range=[0,7])
dist_hist1=np.histogram(dist_Mpc[1][dist_Mpc[1]>0],bins=100,range=[0,7])
dist_hist2=np.histogram(dist_Mpc[2][dist_Mpc[2]>0],bins=100,range=[0,7])
dist_hist3=np.histogram(dist_Mpc[3][dist_Mpc[3]>0],bins=100,range=[0,7])
dist_hist4=np.histogram(dist_Mpc[4][dist_Mpc[4]>0],bins=100,range=[0,7])
bin_center = 0.5*(dist_hist0[1][1]-dist_hist0[1][0])+dist_hist0[1][0:-1]

In [ ]:
plt.figure()
plt.title('cluster profiles at different redshift (for $M_{h}<1.5x10^{14}M_\odot$)')
plt.plot(bin_center,dist_hist0[0]/(Nclus[0]*3.14*bin_center**2),'b',label = 'z<0.5')
plt.plot(bin_center,dist_hist1[0]/(Nclus[1]*3.14*bin_center**2),'c',label = '0.5<z<1.0')
plt.plot(bin_center,dist_hist2[0]/(Nclus[2]*3.14*bin_center**2),'r',label = '1.0<z<1.5')
#plt.errorbar(bin_center,dist_hist0[0]/(Nclus[0]*3.14*bin_center**2),yerr=np.sqrt(dist_hist0[0])/(Nclus[0]*3.14*bin_center**2)) # to add errorbars (negeligeable)
plt.yscale('log')
plt.xscale('log')
plt.ylabel('<Ngal> ($Mpc^{-2}$)');
plt.xlabel('r (Mpc)');
plt.ylim(0.00001,1e4)
plt.xlim(0.03,6)
plt.legend(frameon =False)

plt.figure()
plt.title('cluster profiles at different masses')
plt.plot(bin_center,dist_hist0[0]/(Nclus[0]*3.14*bin_center**2),'b',label = 'z<0.5 & $M_{h}<1.5x10^{14}M_\odot$')
plt.plot(bin_center,dist_hist0[0]/(Nclus[3]*3.14*bin_center**2),'b--',label = 'z<0.5 & $(2<M_{h}<3)x10^{14}M_\odot$')
plt.plot(bin_center,dist_hist1[0]/(Nclus[1]*3.14*bin_center**2),'c',label = '0.5<z<0.1 & $M_{h}<1.5x10^{14}M_\odot$')
plt.plot(bin_center,dist_hist1[0]/(Nclus[4]*3.14*bin_center**2),'c--',label = '0.5<z<0.1 & $(2<M_{h}<3)x10^{14}M_\odot$')
plt.yscale('log')
plt.xscale('log')
plt.ylabel('<Ngal> ($Mpc^{-2}$)');
plt.xlabel('r (Mpc)');
plt.ylim(0.00001,1e4)
plt.xlim(0.03,6)
plt.legend(loc='lower left',frameon =False)

### *At first sight does not look like NFW (or maybe truncated...)*
### *Signal up to r~2-3Mpc*
### *Concentration seems to increase with redshift at fixed mass and amplitude seems to increase with mass at fixed redshift*
### *Need to apply a magnitude cut and plot a model*

# 6 - Galaxy luminosity functions in clusters 

In [ ]:
#compute the composite projected i-band luminosity function for haloes in different mass/z bins

condLF = [(cluster_data['redshift']>0.25)*(cluster_data['redshift']<0.35)*(cluster_data['halo_mass']<1.5e14),\
        (cluster_data['redshift']>0.55)*(cluster_data['redshift']<0.65)*(cluster_data['halo_mass']<1.5e14),\
        (cluster_data['redshift']>0.90)*(cluster_data['redshift']<1.1)*(cluster_data['halo_mass']<1.5e14),\
        (cluster_data['redshift']>1.30)*(cluster_data['redshift']<1.6)*(cluster_data['halo_mass']<1.5e14)]

#halo_listLF = [cluster_data['halo_id'][mhalo*condLF[0]],cluster_data['halo_id'][mhalo*condLF[1]],cluster_data['halo_id'][mhalo*condLF[2]],cluster_data['halo_id'][mhalo*condLF[3]]]
#mag_i = [[],[],[],[]]
#central_mag = [[],[],[],[]]
#bcg_mag = [[],[],[],[]]
#magabs = [[],[],[],[]]
#central_magabs = [[],[],[],[]]
#bcg_magabs = [[],[],[],[]]
#k = 0
#for j in halo_listLF:
#    for halo in j : 
#        central_mag[k].append(cluster_data['mag_i_lsst'][(cluster_data['halo_id']==halo)*(cluster_data['is_central'])][0])
#        central_magabs[k].append(cluster_data['Mag_true_i_lsst_z0'][(cluster_data['halo_id']==halo)*(cluster_data['is_central'])][0])
#        bcg_mag[k].append(np.min(cluster_data['mag_i_lsst'][(cluster_data['halo_id']==halo)]))
#        bcg_magabs[k].append(np.max(cluster_data['Mag_true_i_lsst_z0'][(cluster_data['halo_id']==halo)]))
        
#    k = k+1
    
Nclus = [len(halo_listLF[0]),len(halo_listLF[1]),len(halo_listLF[2]),len(halo_listLF[3])]
print ("Number of clusters in each bin", Nclus)

In [ ]:
#plt.figure()
#plt.hist(np.array(bcg_mag[0])-np.array(central_mag[0]),bins=12,range=[-1.2,0],alpha=0.3,label = '0.25<z<0.35')
#plt.hist(np.array(bcg_mag[1])-np.array(central_mag[1]),bins=12,range=[-1.2,0],alpha=0.3,label = '0.55<z<0.65')
#plt.hist(np.array(bcg_mag[2])-np.array(central_mag[2]),bins=12,range=[-1.2,0],alpha=0.3,label = '0.9<z<0.1')
#plt.hist(np.array(bcg_mag[3])-np.array(central_mag[3]),bins=12,range=[-1.2,0],alpha=0.3,label = '1.4<z<1.5')
#plt.ylabel('counts')
#plt.xlabel('i-mag$_{BCG}$ - i-mag$_{central}$');

In [ ]:
#fraction of halos for which the BCG in i band is not the central in each redshift bin
#for i in range(4):
#    print (np.sum((np.array(bcg_mag[i])-np.array(central_mag[i]))<0)/Nclus[i])


## *The fraction of halos for which the brightest galaxy in the i band is not the central increases with redshift (seems understandable)*

In [ ]:
LF_hist0=np.histogram(cluster_data['mag_i_lsst'][condLF[0]*(~mhalo)],bins=30,range=[15,30])
LF_hist1=np.histogram(cluster_data['mag_i_lsst'][condLF[1]*(~mhalo)],bins=30,range=[15,30])
LF_hist2=np.histogram(cluster_data['mag_i_lsst'][condLF[2]*(~mhalo)],bins=30,range=[15,30])
LF_hist3=np.histogram(cluster_data['mag_i_lsst'][condLF[3]*(~mhalo)],bins=30,range=[15,30])
bin_center = 0.5*(LF_hist0[1][1]-LF_hist0[1][0])+LF_hist0[1][0:-1]

LFa_hist0=np.histogram(cluster_data['Mag_true_i_lsst_z0'][condLF[0]*(~mhalo)],bins=30,range=[-24.5,-13.5])
LFa_hist1=np.histogram(cluster_data['Mag_true_i_lsst_z0'][condLF[1]*(~mhalo)],bins=30,range=[-24.5,-13.5])
LFa_hist2=np.histogram(cluster_data['Mag_true_i_lsst_z0'][condLF[2]*(~mhalo)],bins=30,range=[-24.5,-13.5])
LFa_hist3=np.histogram(cluster_data['Mag_true_i_lsst_z0'][condLF[3]*(~mhalo)],bins=30,range=[-24.5,-13.5])
bin_center_a = 0.5*(LFa_hist0[1][1]-LFa_hist0[1][0])+LFa_hist0[1][0:-1]

In [ ]:
plt.figure()
plt.title('cluster LF at different redshift (for $M_{h}<1.5x10^{14}M_\odot$)')
plt.plot(bin_center,(bin_center[1]-bin_center[0])*LF_hist0[0]/(Nclus[0]),'b.-',label = '0.25<z<0.35')
plt.plot(bin_center,(bin_center[1]-bin_center[0])*LF_hist1[0]/(Nclus[1]),'c.-',label = '0.55<z<0.65')
plt.plot(bin_center,(bin_center[1]-bin_center[0])*LF_hist2[0]/(Nclus[2]),'r.-',label = '0.90<z<1.10')
plt.plot(bin_center,(bin_center[1]-bin_center[0])*LF_hist3[0]/(Nclus[3]),'m.-',label = '1.30<z<1.60')
plt.yscale('log')
plt.ylabel('<Ngal> (mag$^{-1}$)');
plt.xlabel('i-band app mag');
plt.legend(frameon =False)
plt.ylim(0.01,20)

In [ ]:
plt.figure()
plt.title('cluster LF at different redshift (for $M_{h}<1.5x10^{14}M_\odot$)')
plt.plot(bin_center_a,(bin_center_a[1]-bin_center_a[0])*LFa_hist0[0]/(Nclus[0]),'b.-',label = '0.25<z<0.35')
plt.plot(bin_center_a,(bin_center_a[1]-bin_center_a[0])*LFa_hist1[0]/(Nclus[1]),'c.-',label = '0.55<z<0.65')
plt.plot(bin_center_a,(bin_center_a[1]-bin_center_a[0])*LFa_hist2[0]/(Nclus[2]),'r.-',label = '0.90<z<1.10')
plt.plot(bin_center_a,(bin_center_a[1]-bin_center_a[0])*LFa_hist3[0]/(Nclus[3]),'m.-',label = '1.30<z<1.60')
plt.yscale('log')
plt.ylabel('<Ngal> (mag$^{-1}$)');
plt.xlabel('i-band abs mag');
plt.legend(frameon =False)
plt.ylim(0.01,20)

In [ ]:
#test if the central galaxy luminosity distribution is gaussian (null hypothesis taht the distribution is normal)
print ('test in bins of apparent mag')
print (normaltest(central_mag[0]))
print (normaltest(central_mag[1]))
print (normaltest(central_mag[2]))
print (normaltest(central_mag[3]))

print ('test in bins of absolute mag')
print (normaltest(central_magabs[0]))
print (normaltest(central_magabs[1]))
print (normaltest(central_magabs[2]))
print (normaltest(central_magabs[3]))


### *Sharp unexpected decline in the LF at mag fainter than i = ~-18 (see also plot with mag as fct (z) in section 3)* 
### *At very first sight, LF seem OK for galaxies brighter than i ~ -20 (with a shape similar to a double Schechter function), but need more investigation* 
### *Central galaxy luminosity distribution are compatible with being gaussians (as expected)*

# 7 - Color-magnitude diagrams in clusters

In [ ]:
plt.hist2d(cluster_data['Mag_true_g_lsst_z0'],cluster_data['Mag_true_u_lsst_z0']-cluster_data['Mag_true_g_lsst_z0'], bins=50, cmap='PuBu',norm=clr.LogNorm());
plt.plot(cluster_data['Mag_true_g_lsst_z0'][mhalo],cluster_data['Mag_true_u_lsst_z0'][mhalo]-cluster_data['Mag_true_g_lsst_z0'][mhalo],'k.',alpha=0.1)
plt.xlabel('$M_{g}$')
plt.ylabel('$M_{u}-M_{g}$');

In [ ]:
cond = [(cluster_data['redshift']<0.5)*(cluster_data['halo_mass']<1.5e14),\
        (cluster_data['redshift']>0.5)*(cluster_data['redshift']<1)*(cluster_data['halo_mass']<1.5e14),\
        (cluster_data['redshift']>1.0)*(cluster_data['redshift']<1.5)*(cluster_data['halo_mass']<1.5e14),\
        (cluster_data['redshift']<0.5)*(cluster_data['halo_mass']>2e14)*(cluster_data['halo_mass']<3e14),\
        (cluster_data['redshift']>0.5)*(cluster_data['redshift']<1)*(cluster_data['halo_mass']>2e14)*(cluster_data['halo_mass']<3e14),\
       ]
halo_list = [cluster_data['halo_id'][mhalo*cond[0]],cluster_data['halo_id'][mhalo*cond[1]],cluster_data['halo_id'][mhalo*cond[2]],cluster_data['halo_id'][mhalo*cond[3]],cluster_data['halo_id'][mhalo*cond[4]]]
dist_Mpc = [[],[],[],[],[]]
k = 0
Nclus = [len(halo_list[0]),len(halo_list[1]),len(halo_list[2]),len(halo_list[3]),len(halo_list[4])]
for j in halo_list:
    for halo in j :
        ra_ref = cluster_data['ra'][(cluster_data['halo_id']==halo)*mhalo]
        dec_ref = cluster_data['dec'][(cluster_data['halo_id']==halo)*mhalo]
        z_ref = cluster_data['redshift'][(cluster_data['halo_id']==halo)*mhalo]
        dist_Mpc[k].append(cosmo.kpc_proper_per_arcmin(z_ref).value*(np.sqrt((cluster_data['ra'][(cluster_data['halo_id']==halo)]-ra_ref)**2+(cluster_data['dec'][(cluster_data['halo_id']==halo)]-dec_ref)**2))*60./1e3)
    k = k+1
dist_Mpc[0] = np.array(np.hstack(dist_Mpc[0]))
dist_Mpc[1] = np.array(np.hstack(dist_Mpc[1]))
dist_Mpc[2] = np.array(np.hstack(dist_Mpc[2]))
dist_Mpc[3] = np.array(np.hstack(dist_Mpc[3]))
dist_Mpc[4] = np.array(np.hstack(dist_Mpc[4]))

print ("Number of clusters in each bin", Nclus)

In [ ]:
plt.hist2d(cluster_data['Mag_true_g_lsst_z0'][cond[0]],cluster_data['Mag_true_u_lsst_z0'][cond[0]]-cluster_data['Mag_true_g_lsst_z0'][cond[0]], bins=50, cmap='PuBu',norm=clr.LogNorm());
plt.plot(cluster_data['Mag_true_g_lsst_z0'][cond[0]*mhalo],cluster_data['Mag_true_u_lsst_z0'][cond[0]*mhalo]-cluster_data['Mag_true_g_lsst_z0'][cond[0]*mhalo],'k.',alpha=0.2)
plt.xlabel('$M_{g}$')
plt.ylabel('$M_{u}-M_{g}$');


In [ ]:
for i in range(1,4):
    plt.subplot(3, 1, i)
    plt.hist2d(cluster_data['Mag_true_g_lsst_z0'][cond[i-1]],cluster_data['Mag_true_u_lsst_z0'][cond[i-1]]-cluster_data['Mag_true_g_lsst_z0'][cond[i-1]], bins=50, cmap='PuBu',norm=clr.LogNorm());
    plt.plot(cluster_data['Mag_true_g_lsst_z0'][cond[i-1]*mhalo],cluster_data['Mag_true_u_lsst_z0'][cond[i-1]*mhalo]-cluster_data['Mag_true_g_lsst_z0'][cond[i-1]*mhalo],'k.',alpha=0.2)
    plt.xlim(-24.5,-13)
    plt.ylim(0,2)

## *Looks a bit weird, need more investigation*